#Few shot learning (MTL)
[Ref](https://medium.com/analytics-vidhya/multi-task-learning-in-computer-vision-image-classification-ea7b61fb4a0a/)

In [ ]:
from google.colab import drive
drive._mount('/content/drive')  

Mounted at /content/drive


#Part 2: BreaKHis 

In [ ]:
from __future__ import print_function  #do it
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data 
from torch.utils.data import Subset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.optim.lr_scheduler import ExponentialLR, StepLR, ReduceLROnPlateau
from sklearn import metrics
from torch.nn import functional as F
import cv2
import tensorflow as tf


import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

#from mtdp import build_model
#from mtdp.helpers import compute_loss, rescale_head_grads
#from mtdp.loader import MultiImageFolders
#from mtdp.networks import MultiHead

import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)





# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


Training data

In [ ]:
data_dir = '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_train/'
list_of_data = []
img_size = 224      #128 earlier
for fol in os.listdir(data_dir):
    for file in os.listdir(os.path.join(data_dir, fol)):
        json_dict = {}
        img_arr = cv2.imread(os.path.join(os.path.join(data_dir, fol), file))#[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        json_dict['image'] = resized_arr
        json_dict['class_1'] = fol.split('_')[0]
        json_dict['class_2'] = fol.split('_')[1]
        list_of_data.append(json_dict)

x_train = []
y_train = []
print(list_of_data)

for i in range(0, len(list_of_data)):     #train.shape[0]
    x_train.append(list_of_data[i]['image'])
    y_train.append([list_of_data[i]['class_1'], list_of_data[i]['class_2']])

x_train = np.array(x_train) / 255
x_train.reshape(x_train.shape[0],3, img_size, img_size)

print(x_train)

Streaming output truncated to the last 5000 lines.
        [177,  96, 212],
        [177,  96, 212],
        [177,  96, 212]],

       [[210, 195, 218],
        [210, 195, 218],
        [210, 195, 218],
        ...,
        [177,  96, 212],
        [177,  96, 212],
        [177,  96, 212]],

       ...,

       [[132,  41, 199],
        [132,  41, 199],
        [132,  41, 199],
        ...,
        [168,  44, 243],
        [168,  44, 243],
        [168,  44, 243]],

       [[132,  41, 199],
        [132,  41, 199],
        [132,  41, 199],
        ...,
        [168,  44, 243],
        [168,  44, 243],
        [168,  44, 243]],

       [[132,  41, 199],
        [132,  41, 199],
        [132,  41, 199],
        ...,
        [168,  44, 243],
        [168,  44, 243],
        [168,  44, 243]]], dtype=uint8), 'class_1': 'malignant', 'class_2': 'mucinouscarcinoma'}, {'image': array([[[130,  16, 160],
        [130,  16, 160],
        [130,  16, 160],
        ...,
        [181, 100, 194],
     

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=True,  # divide each input by its std
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
y_train = np.array(y_train)
#print(y_train)
class_1 = y_train[:,0]
class_2 = y_train[:,1]
c_1 = len(set(class_1))
c_2 = len(set(class_2))

map_1 = {}
for i, j in enumerate(list(set(class_1))):
    map_1[j] = i

map_2 = {}
for i, j in enumerate(list(set(class_2))):
    map_2[j] = i

class_arr1 = []
for x in class_1:
    class_arr1.append(map_1[x])
class_arr1 = np.array(class_arr1)

class_arr2 = []
for x in class_2:
    class_arr2.append(map_2[x])
class_arr2 = np.array(class_arr2)
print(class_arr1)
print("")
print("-----------")
print("")
print(class_arr2)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

-----------

[0 0 0 0 0 0

Validation data

In [ ]:
val_dir = '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_val/'
list_of_data = []
img_size = 224      #128 earlier
for fol in os.listdir(val_dir):
    for file in os.listdir(os.path.join(val_dir, fol)):
        json_dict = {}
        img_arr = cv2.imread(os.path.join(os.path.join(val_dir, fol), file))#[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        json_dict['image'] = resized_arr
        json_dict['class_1'] = fol.split('_')[0]
        json_dict['class_2'] = fol.split('_')[1]
        list_of_data.append(json_dict)

x_val = []
y_val = []
print(list_of_data)

for i in range(0, len(list_of_data)):     #train.shape[0]
    x_val.append(list_of_data[i]['image'])
    y_val.append([list_of_data[i]['class_1'], list_of_data[i]['class_2']])

x_val = np.array(x_val) / 255
x_val.reshape(x_val.shape[0], 3,img_size, img_size)

print("x_val shape: ", x_val.shape)
print(x_val)

[{'image': array([[[183, 158, 190],
        [183, 158, 190],
        [183, 158, 190],
        ...,
        [175, 149, 189],
        [175, 149, 189],
        [175, 149, 189]],

       [[183, 158, 190],
        [183, 158, 190],
        [183, 158, 190],
        ...,
        [175, 149, 189],
        [175, 149, 189],
        [175, 149, 189]],

       [[183, 158, 190],
        [183, 158, 190],
        [183, 158, 190],
        ...,
        [175, 149, 189],
        [175, 149, 189],
        [175, 149, 189]],

       ...,

       [[185, 155, 204],
        [185, 155, 204],
        [185, 155, 204],
        ...,
        [187, 146, 209],
        [187, 146, 209],
        [187, 146, 209]],

       [[185, 155, 204],
        [185, 155, 204],
        [185, 155, 204],
        ...,
        [187, 146, 209],
        [187, 146, 209],
        [187, 146, 209]],

       [[185, 155, 204],
        [185, 155, 204],
        [185, 155, 204],
        ...,
        [187, 146, 209],
        [187, 146, 209],
        [187,

In [ ]:
y_val = np.array(y_val)
vclass_1 = y_val[:,0]
vclass_2 = y_val[:,1]
vc_1 = len(set(vclass_1))
vc_2 = len(set(vclass_2))

map_1 = {}
for i, j in enumerate(list(set(vclass_1))):
    map_1[j] = i

map_2 = {}
for i, j in enumerate(list(set(vclass_2))):
    map_2[j] = i

val_class_arr1 = []
for x in vclass_1:
    val_class_arr1.append(map_1[x])
val_class_arr1 = np.array(val_class_arr1)

val_class_arr2 = []
for x in vclass_2:
    val_class_arr2.append(map_2[x])
val_class_arr2 = np.array(val_class_arr2)

In [ ]:
from tensorflow import keras
base_model = keras.models.load_model('/content/drive/My Drive/FEW_SHOT_MULTI_TASK/ICIAR_trained/k_iciar_enetb0_m1/model1.h5')    #k_iciar_dnet121_m2 has been saved to ...m1

In [ ]:
from keras.models import Model
model= Model(inputs=base_model.input, outputs=base_model.layers[-3].output) #

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]']      

In [ ]:
import keras 
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules

In [ ]:
import tensorflow_hub as hub
inputs = tf.keras.layers.Input(shape=[224, 224, 3], name='main_input')

main_branch = model(inputs)

#main_branch = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2")(inputs) 
main_branch = tf.keras.layers.Flatten()(main_branch)
main_branch = tf.keras.layers.Dense(1024, activation='relu',  kernel_initializer='he_uniform')(main_branch)   
primary_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='primary_output')(main_branch)
finegrain_branch = tf.keras.layers.Dense(c_2, activation='softmax', name='finegrain_output')(main_branch)      #auto
model = tf.keras.Model(inputs = inputs,
outputs = [primary_branch, finegrain_branch])

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=500,
    decay_rate=0.09)

#opt = keras.optimizers.SGD(learning_rate= 0.001, momentum=1)
opt = keras.optimizers.Adam(learning_rate= 0.01)
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-08)

model.compile(optimizer=opt ,  #rmsprop #sgd
              loss={'primary_output': 'binary_crossentropy', 'finegrain_output': 'sparse_categorical_crossentropy'},
              loss_weights={'primary_output': 2, 'finegrain_output': 5},  metrics=['accuracy'])

model.fit({'main_input': x_train},
          {'primary_output': class_arr1, 'finegrain_output': class_arr2},
          epochs=45, batch_size=16,  #128
          verbose=1,  validation_data = ({'main_input': x_val},
          {'primary_output': val_class_arr1, 'finegrain_output': val_class_arr2}))
          #(validation_split=0.2)

Epoch 1/45
30/30 [==============================] - 34s 410ms/step - loss: 20.5269 - primary_output_loss: 0.7875 - finegrain_output_loss: 3.7904 - primary_output_accuracy: 0.6167 - finegrain_output_accuracy: 0.1521 - val_loss: 32887494.0000 - val_primary_output_loss: 1889502.7500 - val_finegrain_output_loss: 5821698.0000 - val_primary_output_accuracy: 0.5000 - val_finegrain_output_accuracy: 0.1250
Epoch 2/45
30/30 [==============================] - 9s 303ms/step - loss: 10.6952 - primary_output_loss: 0.5433 - finegrain_output_loss: 1.9217 - primary_output_accuracy: 0.7208 - finegrain_output_accuracy: 0.2354 - val_loss: 611396.1250 - val_primary_output_loss: 21197.5039 - val_finegrain_output_loss: 113800.2266 - val_primary_output_accuracy: 0.5000 - val_finegrain_output_accuracy: 0.1250
Epoch 3/45
30/30 [==============================] - 9s 306ms/step - loss: 9.9958 - primary_output_loss: 0.4792 - finegrain_output_loss: 1.8075 - primary_output_accuracy: 0.7937 - finegrain_output_accuracy

In [ ]:
model.save('/content/drive/My Drive/FEW_SHOT_MULTI_TASK/MTL_BreaKHis_trained/mtl2_enetb0_m1/')
#model.save('/content/drive/My Drive/Few_shot_models/fewshot1.h5')

INFO:tensorflow:Assets written to: /content/drive/My Drive/FEW_SHOT_MULTI_TASK/MTL_BreaKHis_trained/mtl2_enetb0_m1/assets


In [ ]:
from tensorflow import keras
loaded_model = keras.models.load_model('/content/drive/My Drive/FEW_SHOT_MULTI_TASK/MTL_BreaKHis_trained/mtl2_enetb0_m1/')

print(loaded_model.layers[0].input_shape)
print()
print("-----------------")
print()
print(model.summary()) 

[(None, 224, 224, 3)]

-----------------

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 1280)         4049571     ['main_input[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 1280)         0           ['model[0][0]']                  
                                                                                                  
 dense (Dense)                  (None, 1024)      

Testing accuracy and loss

In [ ]:
test_dir = '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_test/'
list_of_data = []
img_size = 224      #128 earlier
for fol in os.listdir(test_dir):
    for file in os.listdir(os.path.join(test_dir, fol)):
        json_dict = {}
        img_arr = cv2.imread(os.path.join(os.path.join(test_dir, fol), file))#[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        json_dict['image'] = resized_arr
        json_dict['class_1'] = fol.split('_')[0]
        json_dict['class_2'] = fol.split('_')[1]
        list_of_data.append(json_dict)

x_test = []
y_test = []
print(list_of_data)

for i in range(0, len(list_of_data)):     #train.shape[0]
    x_test.append(list_of_data[i]['image'])
    y_test.append([list_of_data[i]['class_1'], list_of_data[i]['class_2']])

x_test = np.array(x_test) / 255
x_test.reshape(x_test.shape[0], img_size, img_size, 3)

print("x_test shape: ", x_test.shape)
print(x_test)

Streaming output truncated to the last 5000 lines.
        [169, 142, 203],
        [169, 142, 203]],

       [[184, 159, 214],
        [184, 159, 214],
        [184, 159, 214],
        ...,
        [169, 142, 203],
        [169, 142, 203],
        [169, 142, 203]],

       ...,

       [[221, 215, 226],
        [221, 215, 226],
        [221, 215, 226],
        ...,
        [199, 195, 207],
        [199, 195, 207],
        [199, 195, 207]],

       [[221, 215, 226],
        [221, 215, 226],
        [221, 215, 226],
        ...,
        [199, 195, 207],
        [199, 195, 207],
        [199, 195, 207]],

       [[221, 215, 226],
        [221, 215, 226],
        [221, 215, 226],
        ...,
        [199, 195, 207],
        [199, 195, 207],
        [199, 195, 207]]], dtype=uint8), 'class_1': 'malignant', 'class_2': 'lobularcarcinoma'}, {'image': array([[[180, 157, 221],
        [180, 157, 221],
        [180, 157, 221],
        ...,
        [219, 217, 229],
        [219, 217, 229],
      

In [ ]:
y_test = np.array(y_test)
tclass_1 = y_test[:,0]
tclass_2 = y_test[:,1]
tc_1 = len(set(tclass_1))
tc_2 = len(set(tclass_2))

map_1 = {}
for i, j in enumerate(list(set(tclass_1))):
    map_1[j] = i

map_2 = {}
for i, j in enumerate(list(set(tclass_2))):
    map_2[j] = i

test_class_arr1 = []
for x in tclass_1:
    test_class_arr1.append(map_1[x])
test_class_arr1 = np.array(test_class_arr1)

test_class_arr2 = []
for x in tclass_2:
    test_class_arr2.append(map_2[x])
test_class_arr2 = np.array(test_class_arr2)

In [ ]:
results = loaded_model.evaluate({'main_input': x_test},
          {'primary_output': test_class_arr1, 'finegrain_output': test_class_arr2}, batch_size=64)
print("test loss, test acc:", results)

30/30 [==============================] - 11s 253ms/step - loss: 295.3904 - primary_output_loss: 30.9435 - finegrain_output_loss: 46.7007 - primary_output_accuracy: 0.7004 - finegrain_output_accuracy: 0.4568
test loss, test acc: [295.390380859375, 30.943500518798828, 46.70068359375, 0.700426459312439, 0.4568230211734772]
